In [1]:
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt
import scipy.sparse as sps
import scipy.optimize as sco

import import_ipynb
import Fonctions_Test as FT

importing Jupyter notebook from Fonctions_Test.ipynb


# Polynome de degré $n = 2p + 1$ 

## Initialisation

In [2]:
def init_X(p):
    # initialise les points alpha0 et beta0 (3.10) 
    i = np.linspace(0,p,p+1)
    X = np.zeros(2*(p+1))
    X[:p+1] = 1/2*(1-np.cos((2*i+1)*np.pi/(2*p+1)))
    X[p+1:] = 1/2*(1-np.cos(2*i*np.pi/(2*p+1)))
    return X

## Operateur S

In [3]:
def S (p,eps,X0):
    # séparation en alpha et beta
    alpha = X0[:p]
    beta = X0[p:]

    # creation des delta pour alpha et beta
    delta_alpha = np.concatenate((alpha,[1]))
    delta_alpha[1:] -= alpha
    
    delta_beta = - np.concatenate(([0],beta))
    delta_beta[:-1] += beta
    delta_beta[-1] += 1 
    
    # creation des delta_tilde pour alpha et beta
    delta_alpha_t = np.zeros(p+1)
    delta_beta_t = np.zeros(p+1)
    for i in range (p+1):
        delta_alpha_t[i] = max(delta_alpha[i],2*eps)
        delta_beta_t[i] = max(delta_beta[i],2*eps)

    delta_alpha_t = delta_alpha_t / np.sum(delta_alpha_t)
    delta_beta_t = delta_beta_t / np.sum(delta_beta_t)

    # mise a jour des alpha et beta, directement dans x
    X = np.zeros(2*p)
    X[0] = delta_alpha_t[0]
    X[p] = delta_beta_t[0]
    for i in range (1,p):
        X[i] = X[i-1] + delta_alpha_t[i]
        X[i+p] = X[i-1+p] + delta_beta_t[i]
    
    return X

# Algorithme

In [4]:
def Jp(p,f,h,X,epsilon):
    # calcule la jacobienne
    i = np.linspace(0,p-1,p)
    theta_i = 2*(p-i)*np.pi/(2*p+1)
    D_alpha = 2*p*np.cos(p*theta_i)/(np.cos(theta_i)-1) + np.sin(p*theta_i)/np.sin(theta_i) * 2 * (p - 1/(np.cos(theta_i)-1))
    
    i = np.linspace(1,p,p)
    eta_i = (2*(p-i)+1)*np.pi/(2*p+1)
    D_beta = 2*p*np.cos(p*eta_i)/(np.cos(eta_i)+1) + np.sin(p*eta_i)/np.sin(eta_i) * 2* (p + 1/(np.cos(eta_i)+1))
    
    feps = max( np.max(f(X,h)),epsilon) #cf partie 4
    return  np.sqrt(feps) * sps.diags(np.concatenate((D_alpha,D_beta))).toarray()


def b_beta (p,f,h,beta,x,epsilon):
    # calcule la fonction b[beta] au point x
    res = 0
    for i in range (p+1):
        somme = (-1)**(i+p)*np.sqrt(max(f(beta[i],h),epsilon)/(1-beta[i])) 
        pro = 1
        for j in range (p+1):
            if i != j :
                pro *=  (x - beta[j])/(beta[i]-beta[j])
        somme *= pro
        res  += somme
    return res
  

def a_alpha (p,f,h,alpha,x,epsilon):
    # calcule la fonction a[alpha] au point x
    res = 0
    for i in range (p+1):
        somme = (-1)**(i+p)*np.sqrt(max(f(alpha[i],h),epsilon)/(alpha[i])) 
        pro = 1
        for j in range (p+1):
            if i != j :
                pro *=  (x - alpha[j])/(alpha[i]-alpha[j])
        somme *= pro
        res += somme
    return res
     
    
def f_theta(X,p,f,h,epsilon):
    # fonction à minimiser - [a[alpha](beta), b[beta](alpha)]
    alpha = X[:p]
    alpha_1 = np.concatenate((alpha,[1]))
    beta = X[p:]
    beta_0 = np.concatenate(([0],beta))
    return np.concatenate(( b_beta(p,f,h,beta_0,alpha,epsilon), a_alpha(p,f,h,alpha_1,beta,epsilon) ))

In [5]:
def pn(x,ap,bp): 
    #retourne le polynome à partir de la décompo de Lukas
    return x * ap**2 + (1-x) * bp**2

In [6]:
def epsi_S(p):
    #permet de calculer le epsilon propre à l'opérateur Séparateur
    i = np.linspace(0,2*p+1,2*p+2)
    gamma = 1/2*(1-np.cos(i*np.pi/(2*p+1)))
    return min( 1/2/(p+1), np.min((gamma[1:]-gamma[:-1])/3))/2

def Newton_Raphson (X0,p,f,h,epsilon,s=None,eps=1e-12,itermax=100):
    # reconstruit le polynome le plus interpolant
    epsi = epsi_S(p)
    #initialisation
    Xjac = np.copy(X0) 
    Xjac = np.concatenate((Xjac[:p],Xjac[p+2:]))
    X = np.copy(Xjac)
    k=0
    err=2*eps
    err2 = 2*eps
    Residus = []

    while itermax > k and err > eps and err2 > eps :
        #print(f_theta(X,p,f,h,epsilon))
        #print("Cond : ", npl.cond(Jp(p,f,h,X,epsilon)),X)
        d = npl.solve(Jp(p,f,h,X,epsilon),f_theta(X,p,f,h,epsilon))
        X = X - d
        if s != None and s != 0 :
            X = S(p,epsi,X)
        #print(f_theta(X,p,f,h,epsilon))
        k+=1
        X_test = np.concatenate(( X[:p],[1],[0],X[p:]))
        fx = f(X_test,h)
        ap = a_alpha(p,f,h,X_test[:p+1],X_test,epsilon)
        bp = b_beta(p,f,h,X_test[p+1:],X_test,epsilon)
        px = pn(X_test,ap,bp)
        err = npl.norm(fx-px)/npl.norm(fx) 
        err2 = npl.norm(X-Xjac)/npl.norm(Xjac)
        Residus += [err]
        if np.any(1<=X) or np.any(X<=0):
            raise ValueError
            
    return X,k,Residus


def affichage_residus(residus,p,f,h,epsilon):
    plt.figure(0,figsize=(20,5))
    plt.subplot(1,2,1)
    plt.plot(residus)
    name = str(f).split(' ')[1]
    plt.title("Résidus Newton - Fonction "+name+", p = "+str(p)+", h = "+str(h)+", epsilon = "+str(epsilon))
    plt.subplot(1,2,2)
    plt.loglog(residus)
    name = str(f).split(' ')[1]
    plt.title("Log Residus Newton - Fonction "+name+", p = "+str(p)+", h = "+str(h)+", epsilon = "+str(epsilon))
    plt.show()

In [7]:
def approxh(x,p,f,h,epsilon,init=None,s=None,ResidusNewton=None):
    #X = sco.fsolve(f_theta,np.array([1/4,3/4]), args=(p,f,h,epsilon))
    #X = S(p,epsi,X)
    if init is not None and isinstance(init,type(init_X)) : 
        X0 = init(p)
    else :
        X0 = init_X(p)
    X,itermax,residus = Newton_Raphson(X0,p,f,h,epsilon,s)
    if ResidusNewton is not None and ResidusNewton != 0:
        print("Nb iter : \n",len(residus))
        print("Residus :\n",residus[-1])
        if ResidusNewton == 2 : 
            affichage_residus(residus,p,f,h,epsilon)

    X  = np.concatenate(( X[:p],[1],[0],X[p:]))

    ap = a_alpha(p,f,h,X[:p+1],x,epsilon)
    bp = b_beta(p,f,h,X[p+1:],x,epsilon)
    
    return pn(x,ap,bp),[X,a_alpha,b_beta],itermax

### Erreur par les trapèzes

In [8]:
def trapezes(fx,h=None,x=None):
    res = 0
    fx = fx**2
    if x is None :
        x = h*np.linspace(0,1,len(fx))
    for i in range (len(x)-1):
        res += np.abs( (x[i+1]-x[i]) * (fx[i+1]+fx[i])/2 ) 
    return np.sqrt(res)